In [ ]:
import folium
import numpy as np
import utils.basic_utils as bu
from easydict import EasyDict
from tqdm import tqdm
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from folium import Popup
from folium import plugins
# import matplotlib.cm as cm
import branca.colormap as cm
import matplotlib.colors as colors
from math import radians, sin, cos, sqrt, atan2
# from sklearn.metrics.pairwise import haversine_distances
from joblib import Parallel, delayed
import cudf 
from cuml.cluster import DBSCAN
import cupy as cp

import warnings
warnings.filterwarnings("ignore")

name = 'ru'

In [ ]:
df = cudf.read_csv(f'result/stay_points/stay_points_{name}_updatedpub150.csv')
df = df[~df['port_code'].isna()]
df = df[df['min_distance']<10]
df['country_water_body'] = df['country'] + '_' + df['water_body']
labels, uniques = cudf.factorize(df['country_water_body'])
df['cluster_label'] = labels
df.reset_index(drop=True, inplace=True)
len(df['cluster_label'].unique())

328

In [3]:
def compute_distance_matrix_gpu(class_coords):
    """
    给定停靠点坐标数组 class_coords (shape=(n,2), 第一列是 latitude, 第二列是 longitude)，
    利用 GPU（Cupy）计算各点间的 Haversine 距离（单位：千米），返回一个 n x n 的距离矩阵（NumPy 数组）。
    """
    # 将输入转换为 cupy 数组
    coords_cp = cp.asarray(class_coords)  # shape: (n, 2)
    
    # 分别提取纬度和经度。注意：haversine_distance_gpu 的参数顺序是 (lon1, lat1, lon2, lat2)
    lat = coords_cp[:, 0]  # shape: (n,)
    lon = coords_cp[:, 1]  # shape: (n,)
    
    # 将纬度和经度重塑为列向量和行向量以利用广播机制
    lat_i = lat.reshape(-1, 1)   # shape: (n, 1)
    lon_i = lon.reshape(-1, 1)   # shape: (n, 1)
    lat_j = lat.reshape(1, -1)   # shape: (1, n)
    lon_j = lon.reshape(1, -1)   # shape: (1, n)
    
    # 调用你封装好的 GPU Haversine 距离函数，注意传入参数顺序
    distance_matrix_gpu = bu.haversine_distance_gpu(lon_i, lat_i, lon_j, lat_j)
    
    # 将结果从 GPU (cupy 数组) 转换为 NumPy 数组返回
    distance_matrix = cp.asnumpy(distance_matrix_gpu)
    return distance_matrix

In [4]:
# 最大采样数量
max_points = 10000

eps = 200
min_samples = 50

offset = df['cluster_label'].max()
df['cluster'] = df['cluster_label'].copy()

for label in tqdm(df['cluster_label'].to_pandas().unique()):
    mask = df['cluster'] == label
    # 从当前组中获取所有点的经纬度（转换为 pandas 数组）
    class_coords = df[mask][['latitude', 'longitude']].to_pandas().values
    num_points = len(class_coords)
    
    if num_points <= 1:
        df.loc[mask, 'cluster'] = -1
        continue

    # 如果点数超过 max_points，则随机采样
    if num_points > max_points:
        # 随机选取 max_points 个索引，不放回抽样
        sample_idx = np.random.choice(num_points, max_points, replace=False)
        sampled_coords = class_coords[sample_idx]
        # 对采样点计算距离矩阵（分块函数已封装在 compute_distance_matrix_gpu 内）
        distance_matrix_sampled = compute_distance_matrix_gpu(sampled_coords)
        # DBSCAN 聚类（采样点）
        dbscan_cluster_sampled = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed').fit_predict(distance_matrix_sampled)
        # 创建一个全 -1 的聚类结果数组，长度为原组点数
        dbscan_cluster = -1 * np.ones(num_points, dtype=int)
        # 将采样点的聚类结果放入对应位置
        dbscan_cluster[sample_idx] = dbscan_cluster_sampled
    else:
        # 如果点数不超过 max_points，直接计算整个距离矩阵
        distance_matrix = compute_distance_matrix_gpu(class_coords)
        dbscan_cluster = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed').fit_predict(distance_matrix)
    
    # 将 DBSCAN 聚类结果转换为 cupy 数组，并进行调整：
    dbscan_cluster_cp = cp.array(dbscan_cluster)
    # 对于非 -1 的聚类结果，如果为 0 则保留原 cluster，否则加上 offset
    dbscan_cluster_cp = cp.where(dbscan_cluster_cp == -1, -1, cp.where(dbscan_cluster_cp == 0,
                                                    dbscan_cluster_cp,
                                                    dbscan_cluster_cp + offset))
    # 更新原数据中对应点的聚类结果
    df.loc[mask, 'cluster'] = cp.asnumpy(cp.where(dbscan_cluster_cp == -1,
                                                   -1,
                                                   cp.where(dbscan_cluster_cp == 0,
                                                            df.loc[mask, 'cluster'],
                                                            dbscan_cluster_cp)))
    # 更新 offset: 如果本组中 DBSCAN 的最大值为 -1（全部离群），offset 保持不变，否则增加当前组的最大聚类编号
    if dbscan_cluster.max() != -1:
        offset += dbscan_cluster.max()

  0%|          | 0/328 [00:00<?, ?it/s]

100%|██████████| 328/328 [00:11<00:00, 27.58it/s] 


In [ ]:
print(len(df['cluster_label'].unique()))
print(len(df['cluster'].unique()))

df.to_csv(f'result/stay_points/stay_points_{name}_cluster.csv', index=False)

328
266


In [ ]:
### 画离群点图
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=5,tiles='cartodbpositron')
colors = ['blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
for idx, row in df.to_pandas().iterrows():
    color = 'red' if row['cluster'] == -1 else colors[row['cluster'] % len(colors)]
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=2,
        color=color, 
        fill=True,
        # popup=f'Label:{row["cluster"]},Speed_max: {row["speed_max"]}'
        popup=f'Label:{row["cluster"]}'
    ).add_to(m)
m.add_child(folium.LatLngPopup())
m

In [2]:
import cudf
import geopandas as gpd

# 读取并处理数据（已聚类处理后的结果）
df = cudf.read_csv(f'result/stay_points/stay_points_{name}_cluster.csv')
df = df[df['cluster'] != -1]
df['cluster'] = cudf.factorize(df['cluster'])[0]
df = df.to_pandas()

# 1. 将停靠点数据转换为 GeoDataFrame，设置坐标系为 WGS84 (EPSG:4326)
gdf_points = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['longitude'], df['latitude']),
    crs="EPSG:4326"
)

# 2. 按照聚类标签分组，对每个聚类构造凸包作为该聚类的多边形
polygons_list = []
for cluster_label, group in gdf_points.groupby('cluster'):
    if len(group) > 0:
        union_geom = group.unary_union
        convex_hull = union_geom.convex_hull
        ship_cnts = len(group)

        # 合并 country_water_body 字段，按频率降序排序并剔除出现频率小于最大频率一半的元素
        cw_counts = group['country_water_body'].value_counts()
        max_freq_cw = cw_counts.max()  # 最大频率
        cw_counts = cw_counts[cw_counts >= max_freq_cw / 2]  # 只保留频率大于等于最大频率一半的元素
        cw = ', '.join(cw_counts.index.tolist())

        # 合并 nearest_port 字段，按频率降序排序并剔除出现频率小于最大频率一半的元素
        port_counts = group['nearest_port'].value_counts()
        max_freq_ports = port_counts.max()  # 最大频率
        port_counts = port_counts[port_counts >= max_freq_ports / 2]  # 只保留频率大于等于最大频率一半的元素
        ports = ', '.join(port_counts.index.tolist())
        
        polygons_list.append({
            'label': cluster_label,
            'geometry': convex_hull,
            'ship_cnts': ship_cnts,
            'country_water_body': cw,
            'nearest_ports': ports
        })

# 3. 创建一个 GeoDataFrame 保存所有聚类生成的多边形
gdf_polygons = gpd.GeoDataFrame(polygons_list, crs="EPSG:4326")
gdf_polygons.sort_values(by='label', inplace=True)

# 4. 保存结果到 Feather 文件
gdf_polygons.to_feather(f'result/clusters/cl_{name}.feather')
print(f'已保存聚类多边形数据到: result/clusters/cl_{name}.feather')

# 5. 同时保存到 CSV 文件
gdf_polygons.to_csv(f'result/clusters/cl_{name}.csv', index=False)
print(f'已保存聚类多边形数据到: result/clusters/cl_{name}.csv')

已保存聚类多边形数据到: result/clusters/cl_01-03.feather
已保存聚类多边形数据到: result/clusters/cl_01-03.csv
